In [3]:
import zipfile
import pandas as pd

## Unzip Cluster Dataset

In [2]:
# Percorso del file ZIP da estrarre
zip_file_path = "real-time clusters/20230101-20240101_real_time_clusters.parquet.zip"

# Percorso della cartella di destinazione per l'estrazione
extract_to_folder = "real-time clusters"

# Aprire il file ZIP
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Estrarre tutto il contenuto nella cartella di destinazione
    zip_ref.extractall(extract_to_folder)

In [21]:
real_time_clusters_df = pd.read_parquet("real-time clusters/20230101-20240101_real_time_clusters.parquet")

In [28]:
real_time_clusters_df

,alarm_id,network,ne_type,first_occurrence,last_update,data,ne_id,geo_code,min_cluster_first_occurrence,domain,link_cluster,cluster_id,cluster_id2
0,EFM;63af9dfe2aaf2c000125807a,an,lnbts,2022-12-31 03:27:10,2022-12-31 23:59:01,2023-01-01 00:00:02,1btste08d,alad,2022-12-31 03:27:10,mob,None,202301010000_3,202301010000_1
1,EFM;63af9dfec623b40001a5c9cf,an,lnbts,2022-12-31 03:27:10,2022-12-31 23:58:55,2023-01-01 00:00:02,1btste08d,alad,2022-12-31 03:27:10,mob,None,202301010000_3,202301010000_1
2,COS1P;136902468,sdh,ad16,2022-12-31 22:19:34,2022-12-31 23:58:46,2023-01-01 00:00:02,alesitaj/ad16/73,ales,2022-12-31 22:19:34,tx,None,202301010000_20,202301010000_2
3,EFM;63b0beaf2aaf2c0001258736,an,lebts,2022-12-31 23:58:55,2022-12-31 23:59:27,2023-01-01 00:00:02,5btsle119,alss,2022-12-31 23:58:55,mob,None,202301010000_9,202301010000_3
4,COS1P;136891403,sdh,oadm,2022-12-31 11:26:16,2022-12-31 23:59:52,2023-01-01 00:00:02,ancoitab/oadm/1,anco,2022-12-31 11:26:16,tx,None,202301010000_21,202301010000_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,EFMTX;6591efb9d7d6bc00019444d8,adsl,adsl,2023-12-31 23:48:25,2023-12-31 23:56:43,2023-12-31 23:58:02,taglitac/adsl/1,tagl,2023-12-31 23:48:24,tx,None,202312312350_16,202312312350_13
39,EFMTX;6591f046d8beb800016deaef,ptn,ptn3,2023-12-31 23:50:46,2023-12-31 23:57:34,2023-12-31 23:58:02,thesitae/ptn3/1,thes,2023-12-31 23:44:44,tx,None,202312312348_17,202312312344_9
40,EFM;6591f199d4c961000177739d,an,lnbts,2023-12-31 23:56:25,2023-12-31 23:56:37,2023-12-31 23:58:03,1btsts059,trie,2023-12-31 23:56:25,mob,None,202312312358_1,202312312358_13
41,EFM;6591f19aba09b0000160c08d,an,lnbts,2023-12-31 23:56:26,2023-12-31 23:57:59,2023-12-31 23:58:03,1btsts059,trie,2023-12-31 23:56:25,mob,None,202312312358_1,202312312358_13


In [29]:
real_time_clusters_df.columns

Index(['alarm_id', 'network', 'ne_type', 'first_occurrence', 'last_update',
       'data', 'ne_id', 'geo_code', 'min_cluster_first_occurrence', 'domain',
       'link_cluster', 'cluster_id', 'cluster_id2'],
      dtype='object')

## Filter Clusters
- At least 3 alarms inside a cluster
- At least 2 different devices inside a cluster

In [59]:
counts = real_time_clusters_df['cluster_id2'].value_counts()
unique_ne_ids = real_time_clusters_df.groupby('cluster_id2')['ne_id'].nunique()

filtered_df = real_time_clusters_df[
    (real_time_clusters_df['cluster_id2'].isin(counts[counts >= 3].index)) &
    (real_time_clusters_df['cluster_id2'].isin(unique_ne_ids[unique_ne_ids > 1].index))
]

In [63]:
filtered_df.groupby('cluster_id2').size().sort_values(ascending=False)

cluster_id2
202309181126_19    10079
202308080334_40     7989
202311270204_5      6728
202311270406_7      6611
202310110506_13     6226
                   ...  
202309262332_5         3
202309262334_9         3
202305210124_4         3
202305210124_8         3
202309150950_21        3
Length: 624937, dtype: int64

In [64]:
filtered_df.to_parquet("real-time clusters/20230101-20240101_real_time_clusters_filtered.parquet")